In [1]:
# Dependecies
import pandas as pd
import json
from flask import Flask, flash, redirect, render_template, request, session, jsonify, Markup, abort
app = Flask(__name__)


In [2]:
# Read the file
superstore = "Superstore.xls"
superstore_df = pd.read_excel(superstore)

In [3]:
superstore_df.head(2)

,Order Date,Ship Date,Ship Mode,Segment,City,State,Code,Latitude,Longitude,Region,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,2013-11-09,2013-11-12,Second Class,Consumer,Henderson,Kentucky,KY,37.66814,-84.670067,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2013-11-09,2013-11-12,Second Class,Consumer,Henderson,Kentucky,KY,37.66814,-84.670067,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [4]:
Superstore = superstore_df[["City", "State", "Code", "Latitude", "Longitude", "Category", "Sales", "Profit"]]


In [5]:
# SQLAlchemy dependencies 

# To create engine & declarative base 
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
# For data types
from sqlalchemy import Column, Integer, String, Float, Date

# Create base
Base = declarative_base()



In [6]:
Base.metadata.clear()
# Creates Classes which will serve as the anchor points for our Tables
class Orders(Base):
    __table_args__ = {'extend_existing': True}
    __tablename__ = 'Orders'
    index = Column(String, primary_key=True)
    City = Column(String(255),nullable=True)
    State = Column(String(255),nullable=True)
    Code = Column(String(255),nullable=True)
    Latitude = Column(Float,nullable=True)
    Longitude = Column(Float,nullable=True)
    Category = Column(String(255),nullable=True)
    Sales = Column(Float,nullable=True)
    Profit = Column(Float,nullable=True)
    


In [7]:
# Create a sqlite engine
engine = create_engine("sqlite:///superstore.sqlite")

In [8]:
# Add metadata to tables
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [9]:
# Data from Excel to respective tables
Superstore.to_sql('Orders',engine, if_exists='append',index=True)

In [10]:
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Create our session (link) from Python to the DB
session = Session(engine)
results = session.query(Orders.index, Orders.City, Orders.State, Orders.Code, Orders.Latitude, Orders.Longitude, 
                        Orders.Category, Orders.Sales, Orders.Profit).all()
results=json.dumps(results)


In [ ]:
@app.route('/')
def supstr():
  #dat=jsonText
  #return "Hello"
    
    return render_template("index.html",dat=Markup(results))

if __name__ == "__main__":
  app.run(debug=False)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jun/2019 20:14:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2019 20:14:16] "GET /favicon.ico HTTP/1.1" 404 -
